In [0]:


df_bronze = spark.read.parquet("/Volumes/comunicacao/comunicacao/bronze/stream_dados.parquet/")
display(df_bronze)



In [0]:
df_celular = df_bronze.filter(df_bronze['Dispositivo de acesso'] == 'Celular')
display(df_celular)

In [0]:
df_celular = df_celular.withColumnRenamed("Hora da conexão", "Hora_da_conexao")
df_celular.write.format("delta").mode("overwrite").save("/Volumes/comunicacao/comunicacao/bronze/bronze_celular")

In [0]:
df_celular.write.format("delta").mode("overwrite").save("/Volumes/comunicacao/comunicacao/bronze/bronze_celular")